# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [4]:
# !cat publications.tsv

## Import pandas

We are using the very handy pandas library for dataframes.

In [17]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [18]:
publications = pd.read_excel("paper.xlsx")
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2021-11-07,PowerDet:Efficient and Lightweight Object Dete...,CCIS2021,There are problems with the schemes in open el...,"S. Wang, Z. Ou and M. Song, ""PowerDet: Efficie...",PowerDet:Efficient and Lightweight Object Dete...,https://ieeexplore.ieee.org/document/9754678
1,2023-01-18,Development and multi-center validation of mac...,eBioMedicine,Fungal keratitis (FK) is a leading cause of co...,"Wei Z, Wang S, Wang Z, et al. Development and ...",Development and multi-center validation of mac...,https://www.thelancet.com/journals/ebiom/artic...
2,2022-11-14,Deep learning-based classification of infectio...,Therapeutic Advances in Chronic Disease,Infectious keratitis (IK) is an ocular emergen...,"Zhang Z, Wang H, Wang S, et al. Deep learning-...",Deep learning-based classification of infectio...,https://journals.sagepub.com/doi/pdf/10.1177/2...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [19]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [20]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [10]:
!ls ../_publications/

'2021-11-07-PowerDet: Efficient and Lightweight Object Detection for Electric Power Open Scenes.md'


In [13]:
!cat ../_publications/'2021-11-07-PowerDet: Efficient and Lightweight Object Detection for Electric Power Open Scenes.md'

---
title: "PowerDet: Efficient and Lightweight Object Detection for Electric Power Open Scenes"
collection: publications
permalink: /publication/2021-11-07-PowerDet: Efficient and Lightweight Object Detection for Electric Power Open Scenes
excerpt: 'There are problems with the schemes in open electric power scenes, e.g., only able to detect single category, low detection accuracy of multi-scale objects, and difficulty in deploying models on mobile devices. To address the challenges mentioned above, we propose an object detection model, named PowerDet. It is able to detect 9 different types of power facilities efficiently with low cost.'
date: 2021-11-07
venue: 'CCIS2021'
paperurl: 'https://ieeexplore.ieee.org/document/9754678'
citation: 'S. Wang, Z. Ou and M. Song, &quot;PowerDet: Efficient and Lightweight Object Detection for Electric Power Open Scenes,&quot; 2021 IEEE 7th International Conference on Cloud Computing and Intelligent Systems (CCIS), Xi&apos;an, China, 2021, pp. 198-202